<a href="https://colab.research.google.com/github/LucasMoreira7/FinanceAppDatabases/blob/main/Rede_Neural_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Conectando o notebook com o Drive
from google.colab import drive
drive.mount('/content/gdrive')
from google.colab import auth
import gspread

## Importando bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from tensorflow import keras
import numpy as np

# Carregar o DataFrame
data_combined = pd.read_csv('/content/gdrive/MyDrive/FinanceDatabase/BaseCompletaTratada.csv')
display(data_combined)

Mounted at /content/gdrive


,preco,data,despesaOuReceita,parcela,dia_da_semana,Dia_da_Semana,mes,Mes_1,Mes_2,Mes_3,...,categoria_lazer,categoria_moto,categoria_roupa,categoria_sacolao,categoria_salario,categoria_saude,categoria_sem categoria,categoria_supermercado,categoria_telefone,categoria_água
0,19.50,2022-12-20,1,1,Tuesday,1,12,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,10.80,2022-12-20,1,1,Tuesday,1,12,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,9.90,2022-12-20,1,1,Tuesday,1,12,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,19.99,2022-12-20,1,8,Tuesday,1,12,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,43.46,2022-12-20,1,1,Tuesday,1,12,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,25.78,2023-12-12,1,5,Tuesday,1,12,0,0,0,...,0,0,1,0,0,0,0,0,0,0
514,25.78,2024-01-11,1,4,Thursday,3,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
515,25.78,2024-02-10,1,3,Saturday,5,2,0,1,0,...,0,0,1,0,0,0,0,0,0,0
516,25.78,2024-03-11,1,2,Monday,0,3,0,0,1,...,0,0,1,0,0,0,0,0,0,0


Aplicando o modelo de rede neural:

In [2]:
# Definir as variáveis independentes (features) e a variável dependente (target)
X = data_combined.drop(['preco', 'dia_da_semana', 'data'], axis=1).values
y = data_combined['preco'].values

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criar o MinMaxScaler
scaler = MinMaxScaler()

# Ajustar o scaler nos dados de treinamento e transformá-los
X_train_scaled = scaler.fit_transform(X_train)

# Transformar os dados de teste usando o mesmo scaler
X_test_scaled = scaler.transform(X_test)

# Criar o imputer com estratégia 'constant'
imputer = SimpleImputer(strategy='constant', fill_value=1)  # Substituir NaN por 0

# Ajustar o imputer nas features
X_train_imputed = imputer.fit_transform(X_train_scaled)
X_test_imputed = imputer.transform(X_test_scaled)

# Criar o modelo da Rede Neural com camadas adicionais
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))  # Primeira camada adicional
model.add(tf.keras.layers.Dense(128, activation='relu'))  # Segunda camada adicional
model.add(tf.keras.layers.Dense(64, activation='relu'))  # terceira camada adicional
model.add(tf.keras.layers.Dense(32, activation='relu'))  # quarta camada adicional
model.add(tf.keras.layers.Dense(1))  # Camada de saída para previsão numérica

# Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
model.fit(X_train_imputed, y_train, epochs=200, batch_size=14, verbose=1)

# Calcular as previsões da Rede Neural
y_pred_nn = model.predict(X_test_imputed)

# Calcular as métricas de avaliação
mae_nn = mean_absolute_error(y_test, y_pred_nn)
mse_nn = mean_squared_error(y_test, y_pred_nn)
rmse_nn = np.sqrt(mse_nn)
r2_nn = r2_score(y_test, y_pred_nn)

# Exibir as métricas calculadas
print("Mean Absolute Error (RNA):", mae_nn)
print("Mean Squared Error (RNA):", mse_nn)
#print("Root Mean Squared Error (RNA):", rmse_nn)
print("R-squared (RNA):", r2_nn)

Epoch 1/200
26/26 [==============================] - 2s 4ms/step - loss: 84827.1016
Epoch 2/200
26/26 [==============================] - 0s 3ms/step - loss: 81172.7969
Epoch 3/200
26/26 [==============================] - 0s 4ms/step - loss: 74996.9375
Epoch 4/200
26/26 [==============================] - 0s 4ms/step - loss: 72422.8047
Epoch 5/200
26/26 [==============================] - 0s 4ms/step - loss: 69356.5312
Epoch 6/200
26/26 [==============================] - 0s 3ms/step - loss: 65512.9102
Epoch 7/200
26/26 [==============================] - 0s 3ms/step - loss: 60661.1875
Epoch 8/200
26/26 [==============================] - 0s 4ms/step - loss: 54583.9688
Epoch 9/200
26/26 [==============================] - 0s 3ms/step - loss: 48735.2305
Epoch 10/200
26/26 [==============================] - 0s 3ms/step - loss: 41467.7500
Epoch 11/200
26/26 [==============================] - 0s 3ms/step - loss: 35042.3633
Epoch 12/200
26/26 [==============================] - 0s 3ms/step - loss: 

testar depois:

In [ ]:
# Use as médias e desvios padrão das features do conjunto de treinamento
feature_means = np.mean(X_train_imputed, axis=0)
feature_std = np.std(X_train_imputed, axis=0)

# Gere valores fictícios baseados nas distribuições normais
num_samples = 30  # Número de amostras fictícias
fictitious_data = np.random.normal(loc=feature_means, scale=feature_std, size=(num_samples, X_train_imputed.shape[1]))

# Pré-processe os dados fictícios da mesma forma que os dados de treinamento
fictitious_data_scaled = scaler.transform(fictitious_data)
fictitious_data_imputed = imputer.transform(fictitious_data_scaled)

# Faça a previsão usando o modelo treinado
predicted_expenses = model.predict(fictitious_data_imputed)

# Exiba as previsões
#print(predicted_expenses)

Aplicando o modelo de rede neural utilizando k-fold:

In [3]:
# Definir as variáveis independentes (features) e a variável dependente (target)
X = data_combined.drop(['preco', 'dia_da_semana', 'data'], axis=1).values
y = data_combined['preco'].values

# Defina o número de folds (K)
num_folds = 8

# Crie o objeto KFold
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Inicialize uma lista para armazenar os resultados de cada fold
mae_scores = []
mse_scores = []
r2_scores = []

# Loop sobre os folds
for train_index, test_index in kf.split(X):
    # Divida os dados em conjuntos de treinamento e teste usando os índices do KFold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Criar o MinMaxScaler
    scaler = MinMaxScaler()

    # Ajustar o scaler nos dados de treinamento e transformá-los
    X_train_scaled = scaler.fit_transform(X_train)

    # Transformar os dados de teste usando o mesmo scaler
    X_test_scaled = scaler.transform(X_test)

    # Criar o imputer com estratégia 'constant'
    imputer = SimpleImputer(strategy='constant', fill_value=1)  # Substituir NaN por 0

    # Ajustar o imputer nas features
    X_train_imputed = imputer.fit_transform(X_train_scaled)
    X_test_imputed = imputer.transform(X_test_scaled)

    #optimizer = keras.optimizers.Adam(learning_rate=0.001)

    # Criar o modelo da Rede Neural com camadas adicionais
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(32, activation='softplus', input_shape=(X_train_imputed.shape[1],)))
    model.add(tf.keras.layers.Dense(64, activation='softplus'))  # Primeira camada adicional
    model.add(tf.keras.layers.Dense(128, activation='softplus'))  # Segunda camada adicional
    model.add(tf.keras.layers.Dense(64, activation='softplus'))   # Terceira camada adicional
    model.add(tf.keras.layers.Dense(32, activation='softplus'))   # Quarta camada adicional
    model.add(tf.keras.layers.Dense(1))  # Camada de saída para previsão numérica

    # Compilar o modelo
    model.compile(optimizer='adam', loss='mean_squared_error')
    #model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

    # Treinar o modelo
    model.fit(X_train_imputed, y_train, epochs=200, batch_size=32, verbose=1)

    # Calcular as previsões da Rede Neural
    y_pred_nn = model.predict(X_test_imputed)

    # Calcular as métricas de avaliação para este fold
    mae_fold = mean_absolute_error(y_test, y_pred_nn)
    mse_fold = mean_squared_error(y_test, y_pred_nn)
    r2_fold = r2_score(y_test, y_pred_nn)

    # Armazenar as métricas deste fold nas listas
    mae_scores.append(mae_fold)
    mse_scores.append(mse_fold)
    r2_scores.append(r2_fold)

# Calcule as médias das métricas de todos os folds
average_mae = np.mean(mae_scores)
average_mse = np.mean(mse_scores)
average_r2 = np.mean(r2_scores)

print("Average Mean Absolute Error (K-Fold):", average_mae)
print("Average Mean Squared Error (K-Fold):", average_mse)
print("Average R-squared (K-Fold):", average_r2)

Epoch 1/200
15/15 [==============================] - 1s 2ms/step - loss: 86220.6016
Epoch 2/200
15/15 [==============================] - 0s 2ms/step - loss: 81078.8047
Epoch 3/200
15/15 [==============================] - 0s 2ms/step - loss: 74913.7891
Epoch 4/200
15/15 [==============================] - 0s 2ms/step - loss: 74862.7031
Epoch 5/200
15/15 [==============================] - 0s 2ms/step - loss: 74389.1016
Epoch 6/200
15/15 [==============================] - 0s 2ms/step - loss: 74169.7500
Epoch 7/200
15/15 [==============================] - 0s 2ms/step - loss: 74017.9453
Epoch 8/200
15/15 [==============================] - 0s 2ms/step - loss: 73785.3516
Epoch 9/200
15/15 [==============================] - 0s 2ms/step - loss: 73541.8203
Epoch 10/200
15/15 [==============================] - 0s 2ms/step - loss: 73410.2969
Epoch 11/200
15/15 [==============================] - 0s 2ms/step - loss: 73037.5000
Epoch 12/200
15/15 [==============================] - 0s 2ms/step - loss: 

3/3 [==============================] - 0s 4ms/step
Epoch 1/200
15/15 [==============================] - 2s 4ms/step - loss: 70631.8438
Epoch 2/200
15/15 [==============================] - 0s 3ms/step - loss: 66332.3594
Epoch 3/200
15/15 [==============================] - 0s 3ms/step - loss: 61673.8359
Epoch 4/200
15/15 [==============================] - 0s 4ms/step - loss: 61479.8789
Epoch 5/200
15/15 [==============================] - 0s 4ms/step - loss: 61718.7734
Epoch 6/200
15/15 [==============================] - 0s 3ms/step - loss: 61458.3750
Epoch 7/200
15/15 [==============================] - 0s 5ms/step - loss: 61195.6523
Epoch 8/200
15/15 [==============================] - 0s 3ms/step - loss: 61032.6914
Epoch 9/200
15/15 [==============================] - 0s 3ms/step - loss: 61114.6094
Epoch 10/200
15/15 [==============================] - 0s 3ms/step - loss: 60990.3594
Epoch 11/200
15/15 [==============================] - 0s 3ms/step - loss: 60405.1211
Epoch 12/200
15/15 [===

3/3 [==============================] - 0s 5ms/step
Epoch 1/200
15/15 [==============================] - 1s 4ms/step - loss: 91563.7812
Epoch 2/200
15/15 [==============================] - 0s 4ms/step - loss: 88005.8438
Epoch 3/200
15/15 [==============================] - 0s 4ms/step - loss: 81074.0938
Epoch 4/200
15/15 [==============================] - 0s 4ms/step - loss: 79781.4062
Epoch 5/200
15/15 [==============================] - 0s 3ms/step - loss: 79066.5859
Epoch 6/200
15/15 [==============================] - 0s 3ms/step - loss: 78831.5156
Epoch 7/200
15/15 [==============================] - 0s 3ms/step - loss: 78717.2109
Epoch 8/200
15/15 [==============================] - 0s 3ms/step - loss: 78278.2578
Epoch 9/200
15/15 [==============================] - 0s 3ms/step - loss: 78062.2344
Epoch 10/200
15/15 [==============================] - 0s 3ms/step - loss: 77799.8750
Epoch 11/200
15/15 [==============================] - 0s 3ms/step - loss: 77465.0703
Epoch 12/200
15/15 [===